In [ ]:
import time
import numpy as np
import os
import shutil

In [ ]:
arr_dir = "npy"
shard_dir = "shard"

In [ ]:
if os.path.exists(arr_dir):
    print("Deleting exitsting array directory. ")
    shutil.rmtree(arr_dir)
print("(re)creating new array directory")
os.makedirs(arr_dir)

if os.path.exists(shard_dir):
    print("Deleting exitsting shard directory. ")
    shutil.rmtree(shard_dir)
print("(re)creating new shard directory")
os.makedirs(shard_dir)

In [ ]:
num_samples = 100 # number of sample files
X_shape = (640,640)

recipe_fn = "npy_recipe"
npy_recipe = open(recipe_fn, "w")

for i in range(num_samples):
    # create input array
    X = np.ones(X_shape) * i
    # create outpu array
    y = np.array(i)
    
    # set basename for npy input/output files
    arr_fn = "arr_%06d" % i
    # set npy filenames
    X_name = f"{arr_fn}.input.npy"
    y_name = f"{arr_fn}.output.npy"
    # save array
    np.save(os.path.join(arr_dir, X_name), X)
    np.save(os.path.join(arr_dir, y_name), y)
    
    # write sample information onto recipe file
    npy_recipe.write(f"{X_name}\tfile:{os.path.join(arr_dir, X_name)}\n")
    npy_recipe.write(f"{y_name}\tfile:{os.path.join(arr_dir, y_name)}\n")

npy_recipe.close()

 

生成されたinput/outputのnpyファイルをlsコマンドで確認します

In [ ]:
!ls {arr_dir}

レシピファイルの中身を確認します。ここでは`head`コマンドでレシピファイルの先頭部分を表示します

In [ ]:
!head {recipe_fn}

この例では入力ファイルを`{basename}.input.npy`というファイルに、出力（正解）ファイルを`{basename}.output.npy`に出力しています。

その際、レシピファイルでは

{入力ファイル名} file:{入力ファイルまでのファイルパス} <BR>
{出力ファイル名} file:{出力ファイルまでのファイルパス}　<BR>
　・<BR>
　・<BR>
　・<BR>

といった形で出力していきます。


### create tar files using tarp command

`tarp`コマンドを使うと、レシピファイルからwebdataset形式のtarファイルを簡単に作成することができます。 <BR>
https://github.com/webdataset/tarp

In [ ]:
!tarp -v create {recipe_fn} -o npy_webdataset.tar

tarファイルが生成されているか`ls`コマンドで確認します

In [ ]:
ls *.tar

### tarファイルの分割（シャーディング）

大量のファイルをtarファイルにまとめることでデータ転送のオーバーヘッドが改善することができます。
しかし、一つのtarファイルに圧縮してしまうとファイル全体を転送するまで学習を始めることができなくなります。ここでは、tarファイルをいくつかのかたまりに分割する（シャーディング）することを考えます。かたまりのサイズはデータ転送効率の観点から100MB-1GBの間で設定することが多いようです。

シャードされたtarファイルを用いることで、tarファイルをオーバーヘッドなく転送して、なおかつシーケンシャルアクセスにより高速に読み込みます。データセットのシャッフルについては、シャードをシャッフルして読み込むことと、シーケンシャルリードしたシャードをシャッフルすることによって、データセット全体としてのシャッフルを保証します。


tarpを使えば、レシピファイルからシャーディングされたtarファイルを簡単に作成することができます。その場合、`パイプ`を用いることで大元のtarファイルの作成→分割を一つのコマンドラインで記述できます

コマンド中の`-`は標準出力/入力を表します。結果を標準出力に吐き出しそれを次のコマンドの入力としています。tarp splitコマンドでは`-c`オプションでファイルの数ごとにtarファイルを分割します。したがってデータの容量に従って`-c`オプションの数値を適切に調整することになります。ここでは10個のサンプルごとにtarファイルを分割しています（これはトイデータセットなので、このような分割は適切ではありません。あくまで動作を検証するためです）

出力するシャードは`%04d`のような記述をおこなうことで連番のファイルとして出力することが可能です。

In [ ]:
!tarp -v create {recipe_fn} -o - | tarp split - -c 10 -o '{shard_dir}/npy_webdataset-%04d.tar'

作成されたシャード（tarファイル）を確認します。連番のtarファイルが作成されていることを確認します。

In [ ]:
ls {shard_dir}

### upload sharded tar files to S3

作成したWebDataset形式のtarファイルは、ローカルフォルダ上に置いて機械学習のデータセットとして用いることもできます。しかし、WebDatasetが効果を発揮するのは、ファイル転送がボトルネックとなるようなクラウドストレージ上にデータを保存しているときです。

ここでは、作成したtarファイルをS3上にアップロードして動作検証していきます。


In [ ]:
bucket = "put your backet name"
prefix = "put your prefix"
!aws s3 sync --delete {shard_dir} s3://{bucket}/{prefix}/{shard_dir}/